# GFK

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#adaptacion
from utils.adaptacion import gfk_train_all, transform_gfk, gfk_compute_accuracy

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[0]

In [ ]:
def get_source_target_train_data(target_domain, dataset_object):
    X = None
    X_tgt = None
    for domain in dataset_object.domains:
        if target_domain == domain:
            X_tgt = dataset_object.get_all_domain_X(target_domain)
        else:
            if X is None:
                X = dataset_object.get_all_domain_X(domain)
            else:
                X_tmp = dataset_object.get_all_domain_X(domain)
                X = np.concatenate([X, X_tmp])
    
    return X, X_tgt


X, X_tgt = get_source_target_train_data('books', dataset_object)

# Adaptación

## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/amazon/gfk/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en un diccionario separado por dominios, en donde cada dominio contiene una lista con las rutas. El diccionario es almacenado en: models/amazon/gfk/me1_[dimensiones]_models_paths.pkl


## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/amazon/gfk/me1_[dimensiones]_best_models.pkl

# Pruebas con el dataset Amazon (3000 Dimensiones)

In [ ]:
dims = 3000

In [ ]:
print tipo
print dataset_name
print dims
print data_path

In [ ]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

## Creación de modelos de adaptación

In [ ]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_dict = {}

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):    
    print "Cargando rutas de modelos adaptados."
    paths_dict = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parameters = {
        'dims': [int(dims/4), int(dims/2)],
        'n_subs': [10, 20, 50]
    }
    
    print "Creando modelos de adaptacion..."
    for target_domain in domains:
        print target_domain
        X , X_tgt = get_source_target_train_data(target_domain, dataset_object)
        X = X[:, :dims]
        X_tgt = X_tgt[:, :dims]
        
        
        folder_path = os.path.join(models_path, dataset_name, tipo)
        prefix = "me1_%d_%s_" % (dims, target_domain)
        
        new_paths = gfk_train_all(X, X_tgt, parameters, folder_path, prefix)
    
        paths_dict[target_domain] = new_paths
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_dict, models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

In [ ]:
print "Modelos almacenados en:"
for dominio, rutas in paths_dict.items():
    print dominio
    for ruta in rutas:
        print "\t", ruta     

In [ ]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"
else:
    print "Obteniendo mejores modelos..."
    pairs = list(itertools.permutations(domains, 2))
    
    for src, tgt in pairs:
        print "src: %s - tgt: %s" % (src, tgt)
        rutas = paths_dict[tgt]
        
         # se obtienen los datos del dominio
        X_tr = labeled[src]['X_tr'][:, :dims].todense()
        y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
        
        mejor_score = 0
        best_path = None
        for ruta in rutas:
            gfk = joblib.load(ruta)
            X_deep = transform_gfk(X_tr, gfk)
            
            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > mejor_score:
                mejor_score = new_score
                best_path = ruta
        
        # se guarda la ruta del mejor modelo
        best_models[(src, tgt)] = best_path
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

In [ ]:
for dominios, ruta in best_models.items():
    print "%s - %s" % (dominios, ruta)

## Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el par de dominios
    ruta = best_models[(src, tgt)]
    modelo = joblib.load(ruta)

    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts =labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, modelo)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, modelo)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)

    # transfer loss (t)
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

# Pruebas con el dataset Amazon (1000 Dimensiones)

In [ ]:
dims = 1000

In [ ]:
print tipo
print dataset_name
print dims
print data_path

In [ ]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

## Creación de modelos de adaptación

In [ ]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_dict = {}

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):    
    print "Cargando rutas de modelos adaptados."
    paths_dict = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parameters = {
        'dims': [int(dims/4), int(dims/2)],
        'n_subs': [10, 20, 50]
    }
    
    print "Creando modelos de adaptacion..."
    for target_domain in domains:
        print target_domain
        X , X_tgt = get_source_target_train_data(target_domain, dataset_object)
        X = X[:, :dims]
        X_tgt = X_tgt[:, :dims]
        
        
        folder_path = os.path.join(models_path, dataset_name, tipo)
        prefix = "me1_%d_%s_" % (dims, target_domain)
        
        new_paths = gfk_train_all(X, X_tgt, parameters, folder_path, prefix)
    
        paths_dict[target_domain] = new_paths
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_dict, models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

In [ ]:
print "Modelos almacenados en:"
for dominio, rutas in paths_dict.items():
    print dominio
    for ruta in rutas:
        print "\t", ruta     

In [ ]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"
else:
    print "Obteniendo mejores modelos..."
    pairs = list(itertools.permutations(domains, 2))
    
    for src, tgt in pairs:
        print "src: %s - tgt: %s" % (src, tgt)
        rutas = paths_dict[tgt]
        
         # se obtienen los datos del dominio
        X_tr = labeled[src]['X_tr'][:, :dims].todense()
        y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
        
        mejor_score = 0
        best_path = None
        for ruta in rutas:
            gfk = joblib.load(ruta)
            X_deep = transform_gfk(X_tr, gfk)
            
            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > mejor_score:
                mejor_score = new_score
                best_path = ruta
        
        # se guarda la ruta del mejor modelo
        best_models[(src, tgt)] = best_path
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

In [ ]:
for dominios, ruta in best_models.items():
    print "%s - %s" % (dominios, ruta)

## Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el par de dominios
    ruta = best_models[(src, tgt)]
    modelo = joblib.load(ruta)

    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts =labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, modelo)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, modelo)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)

    # transfer loss (t)
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."